In [1]:
from ms3 import Parse
from ms3.utils import transform, roman_numeral2tpc, fifths2pc, fifths2name, name2tpc
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

# Load expanded ABC labels

In [2]:
dir = '/home/hentsche/ABC/harmonies'
p = Parse(dir, key='abc', file_re='tsv$', index='fname')
p.parse_tsv()
p

70 files.
KEY -> EXTENSIONS
-----------------
abc -> {'.tsv': 70}

All 70 tabular files have been parsed, 70 of them as Annotations object(s).
KEY -> ANNOTATION LAYERS
------------------------
abc -> staff  voice  label_type
    -> 1      1      0 (dcml)        117
    ->               3 (dcml)      27908
    ->        2      3 (dcml)          2

In [3]:
labels = p.get_labels().sort_index().iloc[:,:-3]
labels

mc   mn mc_onset mn_onset timesig  staff  voice  label  \
fname                                                                       
n01op18-1_01 0      1    1        0        0     3/4      1      1   .F.I   
             1      2    2        0        0     3/4      1      1      V   
             2      3    3        0        0     3/4      1      1      I   
             3      4    4        0        0     3/4      1      1    IV6   
             4      6    6        0        0     3/4      1      1    V65   
...               ...  ...      ...      ...     ...    ...    ...    ...   
n16op135_04  557  281  281      1/2      1/2     4/4      1      1      I   
             558  281  281      3/4      3/4     4/4      1      1     I6   
             559  282  282        0        0     4/4      1      1  V(64)   
             560  282  282      1/2      1/2     4/4      1      1     V7   
             561  283  283        0        0     4/4      1      1    I\\   

                 globalkey localkey pedal  chord special numeral form figbass  \
fname                                                                           
n01op18-1_01 0           F        I   NaN      I     NaN       I  NaN     NaN   
             1           F        I   NaN      V     NaN       V  NaN     NaN   
             2           F        I   NaN      I     NaN       I  NaN     NaN   
             3           F        I   NaN    IV6     NaN      IV  NaN       6   
             4           F        I   NaN    V65     NaN       V  NaN      65   
...                    ...      ...   ...    ...     ...     ...  ...     ...   
n16op135_04  557         f        I   NaN      I     NaN       I  NaN     NaN   
             558         f        I   NaN     I6     NaN       I  NaN       6   
             559         f        I   NaN  V(64)     NaN       V  NaN     NaN   
             560         f        I   NaN     V7     NaN       V  NaN       7   
             561         f        I   NaN      I     NaN       I  NaN     NaN   

                 changes relativeroot phraseend chord_type  \
fname                                                        
n01op18-1_01 0       NaN          NaN       NaN          M   
             1       NaN          NaN       NaN          M   
             2       NaN          NaN       NaN          M   
             3       NaN          NaN       NaN          M   
             4       NaN          NaN       NaN        Mm7   
...                  ...          ...       ...        ...   
n16op135_04  557     NaN          NaN       NaN          M   
             558     NaN          NaN       NaN          M   
             559      64          NaN       NaN          M   
             560     NaN          NaN       NaN        Mm7   
             561     NaN          NaN        \\          M   

                  globalkey_is_minor  localkey_is_minor    chord_tones  \
fname                                                                    
n01op18-1_01 0                 False              False      (0, 4, 1)   
             1                 False              False      (1, 5, 2)   
             2                 False              False      (0, 4, 1)   
             3                 False              False     (3, 0, -1)   
             4                 False              False  (5, 2, -1, 1)   
...                              ...                ...            ...   
n16op135_04  557                True              False      (0, 4, 1)   
             558                True              False      (4, 1, 0)   
             559                True              False      (1, 0, 4)   
             560                True              False  (1, 5, 2, -1)   
             561                True              False      (0, 4, 1)   

                 added_tones  root  bass_note  
fname                                          
n01op18-1_01 0            ()     0          0  
             1            ()     1          1  
 

# Transform `bass_note` column

## Expressing all bass notes as scale scale degrees of global tonic
Since all scale degrees are expressed as fifths-intervals, this is as easy as adding the local key expressed as fifths

In [4]:
transpose_by = transform(labels, roman_numeral2tpc, ['localkey', 'globalkey_is_minor'])
bass = labels.bass_note + transpose_by
bass.head()

fname          
n01op18-1_01  0    0
              1    1
              2    0
              3    3
              4    5
dtype: Int64

## Adding bass note names to DataFrame

In [5]:
transpose_by = transform(labels, name2tpc, ['globalkey'])
labels['bass_name'] = fifths2name(bass + transpose_by).values
labels.head()

mc  mn mc_onset mn_onset timesig  staff  voice label  \
fname                                                                  
n01op18-1_01 0   1   1        0        0     3/4      1      1  .F.I   
             1   2   2        0        0     3/4      1      1     V   
             2   3   3        0        0     3/4      1      1     I   
             3   4   4        0        0     3/4      1      1   IV6   
             4   6   6        0        0     3/4      1      1   V65   

               globalkey localkey pedal chord special numeral form figbass  \
fname                                                                        
n01op18-1_01 0         F        I   NaN     I     NaN       I  NaN     NaN   
             1         F        I   NaN     V     NaN       V  NaN     NaN   
             2         F        I   NaN     I     NaN       I  NaN     NaN   
             3         F        I   NaN   IV6     NaN      IV  NaN       6   
             4         F        I   NaN   V65     NaN       V  NaN      65   

               changes relativeroot phraseend chord_type  globalkey_is_minor  \
fname                                                                          
n01op18-1_01 0     NaN          NaN       NaN          M               False   
             1     NaN          NaN       NaN          M               False   
             2     NaN          NaN       NaN          M               False   
             3     NaN          NaN       NaN          M               False   
             4     NaN          NaN       NaN        Mm7               False   

                localkey_is_minor    chord_tones added_tones  root  bass_note  \
fname                                                                           
n01op18-1_01 0              False      (0, 4, 1)          ()     0          0   
             1              False      (1, 5, 2)          ()     1          1   
             2              False      (0, 4, 1)          ()     0          0   
             3              False     (3, 0, -1)          ()    -1          3   
             4              False  (5, 2, -1, 1)          ()     1          5   

               bass_name  
fname                     
n01op18-1_01 0         F  
             1         C  
             2         F  
             3         D  
             4         E

## Calculating intervals between successive bass notes
Sloppy version: Include intervals across movement boundaries

### Bass progressions expressed in fifths

In [6]:
bass = bass.bfill()
ivs = bass - bass.shift()
ivs.value_counts()

 0     4809
-1     3918
-5     2903
-2     2746
 5     2537
 2     2490
 1     2061
 3     1576
-4     1269
 4     1055
-3      983
 7      400
 6      332
-7      233
-6      170
 8      147
 9       95
-8       76
-9       68
 10      52
-10      27
 12      27
-12      18
-11       8
-13       6
-14       5
 13       5
 14       4
 11       3
 16       1
 17       1
 15       1
dtype: Int64

### Bass progressions expressed in (enharmonic) semitones

In [7]:
pc_ivs = fifths2pc(ivs)
pc_ivs.index = ivs.index
pc_ivs = pc_ivs.where(pc_ivs <= 6, pc_ivs % -6).fillna(0)
pc_ivs.value_counts()

 0    4855
 5    3927
 1    3303
-2    2803
-1    2771
 2    2521
-5    2074
-3    1645
-4    1416
 4    1132
 3    1078
 6     502
dtype: int64

# Chromatic bass progressions

In [8]:
def cnt(S, interval, k_min=1, df=True):
    ix_chunks = pd.DataFrame(columns=['ixs', 'n']) if df else []
    current = []
    n = 0
    for i, iv in S.iteritems():
        
        if iv in [0, interval]:
            current.append(i)
            if iv == interval:
                n += 1
        else:
            if n >= k_min:
                if df:
                    ix_chunks = ix_chunks.append(pd.Series((current, n), index=['ixs', 'n']), ignore_index=True)
                else:
                    ix_chunks.append((current, n))
            current = [i]
            n = 0
    return ix_chunks

## Successive descending semitones

In [9]:
desc = cnt(pc_ivs, -1)
desc.n.value_counts()

1    2233
2     114
3      62
4      13
5       9
7       3
6       1
Name: n, dtype: int64

### Storing those with three or more

In [10]:
three_desc = labels.loc[desc[desc.n > 2].ixs.sum()]
three_desc.to_csv('three_desc.tsv', sep='\t')
three_desc.head(30)

mc   mn mc_onset mn_onset timesig  staff  voice   label  \
fname                                                                        
n02op18-2_02 160   84   83        0        0     3/4      1      1   V2/IV   
             161   85   84        0        0     3/4      1      1     IV6   
             162   85   84      1/4      1/4     3/4      1      1    ii43   
             163   85   84      1/2      1/2     3/4      1      1   .Ger6   
             164   86   85        0        0     3/4      1      1   V(64)   
             165   86   85      1/2      1/2     3/4      1      1      V7   
n02op18-2_03 81    47   46        0        0     3/4      1      1       I   
             82    48   47        0        0     3/4      1      1     V65   
             83    49   48        0        0     3/4      1      1   V2/IV   
             84    50   49        0        0     3/4      1      1     IV6   
             85    51   50        0        0     3/4      1      1     It6   
             86    52   51        0        0     3/4      1      1       V   
n02op18-2_04 192  160  160      3/8      3/8     2/4      1      1     V65   
             193  161  161      3/8      3/8     2/4      1      1   V2/IV   
             194  162  162        0        0     2/4      1      1     IV6   
             195  162  162      3/8      3/8     2/4      1      1   .Ger6   
             196  163  163        0        0     2/4      1      1     V\\   
             197  163  163      1/8      1/8     2/4      1      1      V7   
             198  164  164        0        0     2/4      1      1   V(64)   
             199  165  165        0        0     2/4      1      1      V7   
             200  166  166        0        0     2/4      1      1   V(64)   
             201  167  167        0        0     2/4      1      1       V   
             202  168  168        0        0     2/4      1      1   V(64)   
             203  169  169        0        0     2/4      1      1       V   
             204  170  170        0        0     2/4      1      1   V(64)   
             205  171  171        0        0     2/4      1      1     V\\   
             206  172  172        0        0     2/4      1      1       V   
             207  175  175        0        0     2/4      1      1   IV.V7   
             208  177  177        0        0     2/4      1      1  V7(+9)   
             209  178  178        0        0     2/4      1      1       V   

                 globalkey localkey pedal   chord special numeral form  \
fname                                                                    
n02op18-2_02 160         C        I   NaN   V2/IV     NaN       V  NaN   
             161         C        I   NaN     IV6     NaN      IV  NaN   
             162         C        I   NaN    ii43     NaN      ii  NaN   
             163         C        I   NaN    Ger6     Ger     vii    o   
             164         C        I   NaN   V(64)     NaN       V  NaN   
             165         C        I   NaN      V7     NaN       V  NaN   
n02op18-2_03 81          G       IV   NaN       I     NaN       I  NaN   
             82          G       IV   NaN     V65     NaN       V  NaN   
             83          G       IV   NaN   V2/IV     NaN       V  NaN   
             84          G       IV   NaN     IV6     NaN      IV  NaN   
             85          G       IV   NaN     It6      It     vii    o   
             86          G       IV   NaN       V     NaN       V  NaN   
n02op18-2_04 192         G       iv   NaN     V65     NaN       V  NaN   
             193         G       iv   NaN   V2/IV     NaN       V  NaN   
             194         G       iv   NaN     IV6     NaN      IV  NaN   
             195         G       iv   NaN    Ger6     Ger     vii    o   
             196         G       iv   NaN       V     NaN       V  NaN   
             197         G       iv   NaN      V7     NaN       V  NaN   
             198         G       iv   NaN 

### Storing those with four or more

In [11]:
four_desc = labels.loc[desc[desc.n > 3].ixs.sum()]
four_desc.to_csv('four_desc.tsv', sep='\t')
four_desc.head(30)

mc   mn mc_onset mn_onset timesig  staff  voice    label  \
fname                                                                         
n02op18-2_03 81    47   46        0        0     3/4      1      1        I   
             82    48   47        0        0     3/4      1      1      V65   
             83    49   48        0        0     3/4      1      1    V2/IV   
             84    50   49        0        0     3/4      1      1      IV6   
             85    51   50        0        0     3/4      1      1      It6   
             86    52   51        0        0     3/4      1      1        V   
n02op18-2_04 192  160  160      3/8      3/8     2/4      1      1      V65   
             193  161  161      3/8      3/8     2/4      1      1    V2/IV   
             194  162  162        0        0     2/4      1      1      IV6   
             195  162  162      3/8      3/8     2/4      1      1    .Ger6   
             196  163  163        0        0     2/4      1      1      V\\   
             197  163  163      1/8      1/8     2/4      1      1       V7   
             198  164  164        0        0     2/4      1      1    V(64)   
             199  165  165        0        0     2/4      1      1       V7   
             200  166  166        0        0     2/4      1      1    V(64)   
             201  167  167        0        0     2/4      1      1        V   
             202  168  168        0        0     2/4      1      1    V(64)   
             203  169  169        0        0     2/4      1      1        V   
             204  170  170        0        0     2/4      1      1    V(64)   
             205  171  171        0        0     2/4      1      1      V\\   
             206  172  172        0        0     2/4      1      1        V   
             207  175  175        0        0     2/4      1      1    IV.V7   
             208  177  177        0        0     2/4      1      1   V7(+9)   
             209  178  178        0        0     2/4      1      1        V   
n03op18-3_04 288  200  200        0        0     6/8      1      1      I64   
             289  201  201        0        0     6/8      1      1  I.viio7   
             290  203  203        0        0     6/8      1      1   .bVII7   
             291  205  205        0        0     6/8      1      1     ii64   
             292  207  207        0        0     6/8      1      1  V2/bIII   
             293  209  209        0        0     6/8      1      1       V7   

                 globalkey localkey pedal    chord special numeral form  \
fname                                                                     
n02op18-2_03 81          G       IV   NaN        I     NaN       I  NaN   
             82          G       IV   NaN      V65     NaN       V  NaN   
             83          G       IV   NaN    V2/IV     NaN       V  NaN   
             84          G       IV   NaN      IV6     NaN      IV  NaN   
             85          G       IV   NaN      It6      It     vii    o   
             86          G       IV   NaN        V     NaN       V  NaN   
n02op18-2_04 192         G       iv   NaN      V65     NaN       V  NaN   
             193         G       iv   NaN    V2/IV     NaN       V  NaN   
             194         G       iv   NaN      IV6     NaN      IV  NaN   
             195         G       iv   NaN     Ger6     Ger     vii    o   
             196         G       iv   NaN        V     NaN       V  NaN   
             197         G       iv   NaN       V7     NaN       V  NaN   
             198         G       iv   NaN    V(64)     NaN       V  NaN   
             199         G       iv   NaN       V7     NaN       V  NaN   
             200         G       iv   NaN    V(64)     NaN       V  NaN   
             201         G       iv   NaN        V     NaN       V  NaN   
             202         G       iv   NaN    V(64)     NaN       V  NaN   
             203         G       iv   NaN        V     NaN       

## Successive ascending semitones

In [12]:
asc = cnt(pc_ivs, 1)
asc.n.value_counts()

1     2446
2      202
3       98
5       14
4        9
6        3
7        2
13       1
8        1
Name: n, dtype: int64

### Storing those with three or more

In [13]:
three_asc = labels.loc[asc[asc.n > 2].ixs.sum()]
three_asc.to_csv('three_asc.tsv', sep='\t')
three_asc.head(30)

mc   mn mc_onset mn_onset timesig  staff  voice  \
fname                                                                
n01op18-1_01 167  136  136      1/4      1/4     3/4      1      1   
             168  137  137        0        0     3/4      1      1   
             169  137  137      1/4      1/4     3/4      1      1   
             170  137  137      1/2      1/2     3/4      1      1   
             171  138  138        0        0     3/4      1      1   
             172  138  138      1/2      1/2     3/4      1      1   
             181  143  143        0        0     3/4      1      1   
             182  143  143      1/4      1/4     3/4      1      1   
             183  143  143      1/2      1/2     3/4      1      1   
             184  144  144        0        0     3/4      1      1   
n01op18-1_02 48    19   19        0        0     9/8      1      1   
             49    19   19      3/8      3/8     9/8      1      1   
             50    19   19      3/4      3/4     9/8      1      1   
             51    20   20        0        0     9/8      1      1   
             52    21   21        0        0     9/8      1      1   
             53    21   21      7/8      7/8     9/8      1      1   
n01op18-1_04 13     7    7      1/8      1/8     2/4      1      1   
             14     7    7      1/4      1/4     2/4      1      1   
             15     7    7      3/8      3/8     2/4      1      1   
             16     8    8        0        0     2/4      1      1   
             17     8    8      1/8      1/8     2/4      1      1   
             32    15   15      1/8      1/8     2/4      1      1   
             33    15   15      1/4      1/4     2/4      1      1   
             34    15   15      3/8      3/8     2/4      1      1   
             35    16   16        0        0     2/4      1      1   
             168   97   97      1/8      1/8     2/4      1      1   
             169   97   97      1/4      1/4     2/4      1      1   
             170   97   97      3/8      3/8     2/4      1      1   
             171   98   98        0        0     2/4      1      1   
             172   98   98      1/8      1/8     2/4      1      1   

                       label globalkey localkey pedal       chord special  \
fname                                                                       
n01op18-1_01 167       @none         F       ii   NaN       @none     NaN   
             168      #viio7         F       ii   NaN      #viio7     NaN   
             169         VI6         F       ii   NaN         VI6     NaN   
             170      V65/ii         F       ii   NaN      V65/ii     NaN   
             171     #viio65         F       ii   NaN     #viio65     NaN   
             172          i6         F       ii   NaN          i6     NaN   
             181      #viio7         F        v   NaN      #viio7     NaN   
             182         VI6         F        v   NaN         VI6     NaN   
             183    #viio/ii         F        v   NaN    #viio/ii     NaN   
             184         V43         F        v   NaN         V43     NaN   
n01op18-1_02 48   V7(4)/bVII         d      III   NaN  V7(4)/bVII     NaN   
             49      V7/bVII         d      III   NaN     V7/bVII     NaN   
             50     #viio7/V         d      III   NaN    #viio7/V     NaN   
             51          V\\         d      III   NaN           V     NaN   
             52          iv6         d      III   NaN         iv6     NaN   
             53        .Ger6         d      III   NaN        Ger6     Ger   
n01op18-1_04 13          V65         F        I   NaN         V65     NaN   
             14         V.IV         F        V   NaN          IV     NaN   
             15        V65/V         F        V   NaN       V65/V     NaN   
             16        V(64)         F        V   NaN       V(64)     NaN   
             17           V7         F        V   NaN          V7     NaN   
       

### Storing those with four or more

In [14]:
four_asc = labels.loc[asc[asc.n > 3].ixs.sum()]
four_asc.to_csv('four_asc.tsv', sep='\t')
four_asc.head(30)

mc   mn mc_onset mn_onset timesig  staff  voice      label  \
fname                                                                           
n01op18-1_01 167  136  136      1/4      1/4     3/4      1      1      @none   
             168  137  137        0        0     3/4      1      1     #viio7   
             169  137  137      1/4      1/4     3/4      1      1        VI6   
             170  137  137      1/2      1/2     3/4      1      1     V65/ii   
             171  138  138        0        0     3/4      1      1    #viio65   
             172  138  138      1/2      1/2     3/4      1      1         i6   
n02op18-2_03 59    38   37        0        0     3/4      1      1         I6   
             60    38   37      3/8      3/8     3/4      1      1        I+6   
             61    38   37      1/2      1/2     3/4      1      1         IV   
             62    38   37      5/8      5/8     3/4      1      1      V65/V   
             63    39   38        0        0     3/4      1      1      V(64)   
             64    39   38      1/8      1/8     3/4      1      1  #viio7/vi   
             65    39   38      1/4      1/4     3/4      1      1         vi   
n03op18-3_01 252  174  174        0        0     2/2      1      1        IV6   
             253  175  175        0        0     2/2      1      1  #viio65/V   
             254  176  176        0        0     2/2      1      1         v6   
             255  177  177        0        0     2/2      1      1        V65   
             256  178  178        0        0     2/2      1      1        vi6   
             257  179  179        0        0     2/2      1      1  #viio7/ii   
             258  180  180        0        0     2/2      1      1     viio65   
             338  237  237        0        0     2/2      1      1          I   
             339  238  238        0        0     2/2      1      1   .bIII.v6   
             340  239  239        0        0     2/2      1      1         V6   
             341  240  240        0        0     2/2      1      1          I   
             342  241  241        0        0     2/2      1      1     iv.V65   
             343  243  243        0        0     2/2      1      1          i   
n03op18-3_02 85    31   31      1/4      1/4     2/4      1      1    V65/III   
             86    32   32      1/4      1/4     2/4      1      1        III   
             87    32   32      3/8      3/8     2/4      1      1     V65/iv   
             88    33   33        0        0     2/4      1      1         iv   

                 globalkey localkey pedal      chord special numeral form  \
fname                                                                       
n01op18-1_01 167         F       ii   NaN      @none     NaN   @none  NaN   
             168         F       ii   NaN     #viio7     NaN    #vii    o   
             169         F       ii   NaN        VI6     NaN      VI  NaN   
             170         F       ii   NaN     V65/ii     NaN       V  NaN   
             171         F       ii   NaN    #viio65     NaN    #vii    o   
             172         F       ii   NaN         i6     NaN       i  NaN   
n02op18-2_03 59          G        I   NaN         I6     NaN       I  NaN   
             60          G        I   NaN        I+6     NaN       I    +   
             61          G        I   NaN         IV     NaN      IV  NaN   
             62          G        I   NaN      V65/V     NaN       V  NaN   
             63          G        I   NaN      V(64)     NaN       V  NaN   
             64          G        I   NaN  #viio7/vi     NaN    #vii    o   
             65          G        I   NaN         vi     NaN      vi  NaN   
n03op18-3_01 252         D        I   NaN        IV6     NaN      IV  NaN   
             253         D        I   NaN  #viio65/V     NaN    #vii    o   
             254         D        I   NaN         v6     NaN       v  NaN   
             255         D        I   Na

# Filtering for particular progressions with length >= 3
Finding only direct successors

In [15]:
def filtr(df, query, column='chord'):
    vals = df[column].to_list()
    n_grams = [t for t in zip(*(vals[i:] for i in range(len(query))))]
    if isinstance(query[0], str):
        lengths = [len(q) for q in query]
        n_grams = [tuple(e[:l] for e,l  in zip(t, lengths)) for t in n_grams]
    return query in n_grams

def show(df, query, column='chord'):
    selector = df.groupby(level=0).apply(filtr, query, column)
    return df[selector[df.index.get_level_values(0)].values]

## Descending

In [16]:
descending = pd.concat([labels.loc[ix_seq] for ix_seq in desc[desc.n > 2].ixs.values], keys=range((desc.n > 2).sum()))
descending

mc  mn mc_onset mn_onset timesig  staff  voice   label  \
   fname                                                                      
0  n02op18-2_02 160  84  83        0        0     3/4      1      1   V2/IV   
                161  85  84        0        0     3/4      1      1     IV6   
                162  85  84      1/4      1/4     3/4      1      1    ii43   
                163  85  84      1/2      1/2     3/4      1      1   .Ger6   
                164  86  85        0        0     3/4      1      1   V(64)   
...                  ..  ..      ...      ...     ...    ...    ...     ...   
87 n16op135_03  157  47  47        0        0     6/8      1      1  V65/vi   
                158  47  47      3/8      3/8     6/8      1      1   V2/ii   
                159  48  48        0        0     6/8      1      1   V65/V   
                160  48  48      3/8      3/8     6/8      1      1      V2   
                161  49  49      3/8      3/8     6/8      1      1      I6   

                    globalkey localkey pedal   chord special numeral form  \
   fname                                                                    
0  n02op18-2_02 160         C        I   NaN   V2/IV     NaN       V  NaN   
                161         C        I   NaN     IV6     NaN      IV  NaN   
                162         C        I   NaN    ii43     NaN      ii  NaN   
                163         C        I   NaN    Ger6     Ger     vii    o   
                164         C        I   NaN   V(64)     NaN       V  NaN   
...                       ...      ...   ...     ...     ...     ...  ...   
87 n16op135_03  157        Db        I   NaN  V65/vi     NaN       V  NaN   
                158        Db        I   NaN   V2/ii     NaN       V  NaN   
                159        Db        I   NaN   V65/V     NaN       V  NaN   
                160        Db        I   NaN      V2     NaN       V  NaN   
                161        Db        I   NaN      I6     NaN       I  NaN   

                    figbass changes relativeroot phraseend chord_type  \
   fname                                                                
0  n02op18-2_02 160       2     NaN           IV       NaN        Mm7   
                161       6     NaN          NaN       NaN          M   
                162      43     NaN          NaN       NaN        mm7   
                163      65      b3            V       NaN        Ger   
                164     NaN      64          NaN       NaN          M   
...                     ...     ...          ...       ...        ...   
87 n16op135_03  157      65     NaN           vi       NaN        Mm7   
                158       2     NaN           ii       NaN        Mm7   
                159      65     NaN            V       NaN        Mm7   
                160       2     NaN          NaN       NaN        Mm7   
                161       6     NaN          NaN       NaN          M   

                     globalkey_is_minor  localkey_is_minor     chord_tones  \
   fname                                                                     
0  n02op18-2_02 160               False              False   (-2, 0, 4, 1)   
                161               False              False      (3, 0, -1)   
                162               False              False   (3, 0, 2, -1)   
                163               False              False  (-4, 0, -3, 6)   
                164               False              False       (1, 0, 4)   
...                                 ...                ...             ...   
87 n16op135_03  157               False              False    (8, 5, 2, 4)   
                158               False              False    (1, 3, 7, 4)   
                159               False              False    (6, 3, 0, 2)   
                160               False              False   (-1, 1, 5, 2)   
                161               False              False       (4, 1, 0)   

                    added_

### Looking for `Ger i64`

In [17]:
show(descending, ('Ger', 'i64'))

,,,mc,mn,mc_onset,mn_onset,timesig,staff,voice,label,globalkey,localkey,pedal,chord,special,numeral,form,figbass,changes,relativeroot,phraseend,chord_type,globalkey_is_minor,localkey_is_minor,chord_tones,added_tones,root,bass_note,bass_name
,fname,,,,,,,,,,,,,,,,,,,,,,,,,,,,


### `i64`

In [24]:
show(descending, ('i64',))

mc   mn mc_onset mn_onset timesig  staff  voice  \
   fname                                                                
6  n03op18-3_04 288  200  200        0        0     6/8      1      1   
                289  201  201        0        0     6/8      1      1   
                290  203  203        0        0     6/8      1      1   
                291  205  205        0        0     6/8      1      1   
                292  207  207        0        0     6/8      1      1   
                293  209  209        0        0     6/8      1      1   
                294  211  211        0        0     6/8      1      1   
                295  213  213        0        0     6/8      1      1   
                296  215  215        0        0     6/8      1      1   
20 n06op18-6_04 6      7    7        0        0     2/4      1      1   
                7      8    8        0        0     2/4      1      1   
                8      8    8      1/4      1/4     2/4      1      1   
                9      9    9        0        0     2/4      1      1   
                10    10   10        0        0     2/4      1      1   
                11    10   10    15/32    15/32     2/4      1      1   
                12    11   11        0        0     2/4      1      1   
                13    11   11     7/32     7/32     2/4      1      1   
                14    11   11      1/4      1/4     2/4      1      1   
                15    12   12        0        0     2/4      1      1   
32 n08op59-2_01 345  192  190     9/16     9/16     6/8      1      1   
                346  192  190      5/8      5/8     6/8      1      1   
                347  193  191        0        0     6/8      1      1   
                348  193  191      1/2      1/2     6/8      1      1   

                         label globalkey localkey pedal      chord special  \
   fname                                                                     
6  n03op18-3_04 288        I64         D       IV   NaN        I64     NaN   
                289    I.viio7         D        I   NaN      viio7     NaN   
                290     .bVII7         D        I   NaN      bVII7     NaN   
                291       ii64         D        I   NaN       ii64     NaN   
                292    V2/bIII         D        I   NaN    V2/bIII     NaN   
                293         V7         D        I   NaN         V7     NaN   
                294        i64         D        I   NaN        i64     NaN   
                295       V(4)         D        I   NaN       V(4)     NaN   
                296         V7         D        I   NaN         V7     NaN   
20 n06op18-6_04 6            I        Bb        I   NaN          I     NaN   
                7        V6(6)        Bb        I   NaN      V6(6)     NaN   
                8           V6        Bb        I   NaN         V6     NaN   
                9    #viio2/ii        Bb        I   NaN  #viio2/ii     NaN   
                10       V7/ii        Bb        I   NaN      V7/ii     NaN   
                11    V(64)/ii        Bb        I   NaN   V(64)/ii     NaN   
                12       V7/ii        Bb        I   NaN      V7/ii     NaN   
                13    V(64)/ii        Bb        I   NaN   V(64)/ii     NaN   
                14       V7/ii        Bb        I   NaN      V7/ii     NaN   
                15      #i.i64        Bb       #i   NaN        i64     NaN   
32 n08op59-2_01 345        i64         e        i   NaN        i64     NaN   
                346   #viio7/V         e        i   NaN   #viio7/V     NaN   
                347    #viio43         e        i   NaN    #viio43     NaN   
                348       I.I6         e        I   NaN         I6     NaN   

                    numeral form figbass changes relativeroot phraseend  \
   fname                                                                  
6  n03op18-3_04 288       I  NaN      64     NaN          NaN       NaN   
                289 

### `Ger V(64)`

In [18]:
show(descending, ('Ger', 'V(64'))

mc   mn mc_onset mn_onset timesig  staff  voice  \
   fname                                                                
0  n02op18-2_02 160   84   83        0        0     3/4      1      1   
                161   85   84        0        0     3/4      1      1   
                162   85   84      1/4      1/4     3/4      1      1   
                163   85   84      1/2      1/2     3/4      1      1   
                164   86   85        0        0     3/4      1      1   
                165   86   85      1/2      1/2     3/4      1      1   
63 n13op130_01  506  161  160        0        0     4/4      1      1   
                507  161  160      1/4      1/4     4/4      1      1   
                508  161  160      1/2      1/2     4/4      1      1   
                509  161  160      3/4      3/4     4/4      1      1   
                510  162  161        0        0     4/4      1      1   
                511  162  161      1/4      1/4     4/4      1      1   
                512  162  161      1/2      1/2     4/4      1      1   
77 n15op132_01  39    20   20        0        0     4/4      1      1   
                40    20   20      1/4      1/4     4/4      1      1   
                41    20   20      1/2      1/2     4/4      1      1   
                42    20   20      3/4      3/4     4/4      1      1   
                43    21   21        0        0     4/4      1      1   
78 n15op132_01  353  156  156      3/4      3/4     4/4      1      1   
                354  156  156    15/16    15/16     4/4      1      1   
                355  157  157     3/16     3/16     4/4      1      1   
                356  157  157     7/16     7/16     4/4      1      1   

                          label globalkey localkey pedal    chord special  \
   fname                                                                    
0  n02op18-2_02 160       V2/IV         C        I   NaN    V2/IV     NaN   
                161         IV6         C        I   NaN      IV6     NaN   
                162        ii43         C        I   NaN     ii43     NaN   
                163       .Ger6         C        I   NaN     Ger6     Ger   
                164       V(64)         C        I   NaN    V(64)     NaN   
                165          V7         C        I   NaN       V7     NaN   
63 n13op130_01  506           V        Bb     bIII   NaN        V     NaN   
                507     #viio/V        Bb     bIII   NaN  #viio/V     NaN   
                508          V2        Bb     bIII   NaN       V2     NaN   
                509      I6(+6)        Bb     bIII   NaN   I6(+6)     NaN   
                510  .bVII.Ger6        Bb     bVII   NaN     Ger6     Ger   
                511       V(64)        Bb     bVII   NaN    V(64)     NaN   
                512     #viio/V        Bb     bVII   NaN  #viio/V     NaN   
77 n15op132_01  39        #viio         a        i   NaN    #viio     NaN   
                40        V2/iv         a        i   NaN    V2/iv     NaN   
                41          IV6         a        i   NaN      IV6     NaN   
                42        .Ger6         a        i   NaN     Ger6     Ger   
                43        V(64)         a        i   NaN    V(64)     NaN   
78 n15op132_01  353       V2/IV         a      III   NaN    V2/IV     NaN   
                354         IV6         a      III   NaN      IV6     NaN   
                355       .Ger6         a      III   NaN     Ger6     Ger   
                356       V(64)         a      III   NaN    V(64)     NaN   

                    numeral form figbass changes relativeroot phraseend  \
   fname                                                                  
0  n02op18-2_02 160       V  NaN       2     NaN           IV       NaN   
                161      IV  NaN       6     NaN          NaN       NaN   
                162      ii  NaN      43     NaN          NaN       NaN   
                163     vii    o      65     

### Bass degrees `b6 5 #4`

In [19]:
show(descending, (-4, 1, 6), 'bass_note')

mc   mn mc_onset mn_onset timesig  staff  voice  \
   fname                                                               
54 n12op127_02 335   77   77      7/8      7/8     2/2      1      1   
               336   78   78        0        0     2/2      1      1   
               337   78   78      1/4      1/4     2/2      1      1   
               338   78   78      3/8      3/8     2/2      1      1   
               339   78   78      1/2      1/2     2/2      1      1   
               340   78   78      3/4      3/4     2/2      1      1   
               341   78   78      7/8      7/8     2/2      1      1   
               342   79   79        0        0     2/2      1      1   
               343   79   79      1/8      1/8     2/2      1      1   
63 n13op130_01 506  161  160        0        0     4/4      1      1   
               507  161  160      1/4      1/4     4/4      1      1   
               508  161  160      1/2      1/2     4/4      1      1   
               509  161  160      3/4      3/4     4/4      1      1   
               510  162  161        0        0     4/4      1      1   
               511  162  161      1/4      1/4     4/4      1      1   
               512  162  161      1/2      1/2     4/4      1      1   

                         label globalkey localkey pedal     chord special  \
   fname                                                                    
54 n12op127_02 335          V6        Ab       #V   NaN        V6     NaN   
               336     I[V2/IV        Ab       #V     I     V2/IV     NaN   
               337         IV6        Ab       #V     I       IV6     NaN   
               338         iv6        Ab       #V     I       iv6     NaN   
               339      I64(4)        Ab       #V     I    I64(4)     NaN   
               340    V65(2)/V        Ab       #V     I  V65(2)/V     NaN   
               341       V65/V        Ab       #V     I     V65/V     NaN   
               342       ii%65        Ab       #V     I     ii%65     NaN   
               343          I6        Ab       #V     I        I6     NaN   
63 n13op130_01 506           V        Bb     bIII   NaN         V     NaN   
               507     #viio/V        Bb     bIII   NaN   #viio/V     NaN   
               508          V2        Bb     bIII   NaN        V2     NaN   
               509      I6(+6)        Bb     bIII   NaN    I6(+6)     NaN   
               510  .bVII.Ger6        Bb     bVII   NaN      Ger6     Ger   
               511       V(64)        Bb     bVII   NaN     V(64)     NaN   
               512     #viio/V        Bb     bVII   NaN   #viio/V     NaN   

                   numeral form figbass changes relativeroot phraseend  \
   fname                                                                 
54 n12op127_02 335       V  NaN       6     NaN          NaN       NaN   
               336       V  NaN       2     NaN           IV       NaN   
               337      IV  NaN       6     NaN          NaN       NaN   
               338      iv  NaN       6     NaN          NaN       NaN   
               339       I  NaN      64       4          NaN       NaN   
               340       V  NaN      65       2            V       NaN   
               341       V  NaN      65     NaN            V       NaN   
               342      ii    %      65     NaN          NaN       NaN   
               343       I  NaN       6     NaN          NaN       NaN   
63 n13op130_01 506       V  NaN     NaN     NaN          NaN       NaN   
               507    #vii    o     NaN     NaN            V       NaN   
               508       V  NaN       2     NaN          NaN       NaN   
               509       I  NaN       6      +6          NaN       NaN   
               510     vii    o      65      b3            V       NaN   
               511       V  NaN     NaN      64          NaN       NaN   
               512    #vii    o     NaN     NaN            V       NaN   

    

## Ascending

In [20]:
ascending = pd.concat([labels.loc[ix_seq] for ix_seq in asc[asc.n > 2].ixs.values], keys=range((asc.n > 2).sum()))
ascending = ascending[ascending.label != '@none']
ascending

mc   mn mc_onset mn_onset timesig  staff  voice  \
    fname                                                                
0   n01op18-1_01 168  137  137        0        0     3/4      1      1   
                 169  137  137      1/4      1/4     3/4      1      1   
                 170  137  137      1/2      1/2     3/4      1      1   
                 171  138  138        0        0     3/4      1      1   
                 172  138  138      1/2      1/2     3/4      1      1   
...                   ...  ...      ...      ...     ...    ...    ...   
127 n15op132_05  703  396  396        0        0     3/4      1      1   
                 704  396  396      1/2      1/2     3/4      1      1   
                 705  397  397        0        0     3/4      1      1   
                 706  397  397      1/4      1/4     3/4      1      1   
                 707  398  398        0        0     3/4      1      1   

                        label globalkey localkey pedal    chord special  \
    fname                                                                 
0   n01op18-1_01 168   #viio7         F       ii   NaN   #viio7     NaN   
                 169      VI6         F       ii   NaN      VI6     NaN   
                 170   V65/ii         F       ii   NaN   V65/ii     NaN   
                 171  #viio65         F       ii   NaN  #viio65     NaN   
                 172       i6         F       ii   NaN       i6     NaN   
...                       ...       ...      ...   ...      ...     ...   
127 n15op132_05  703    vio64         a        I   NaN    vio64     NaN   
                 704     vi64         a        I   NaN     vi64     NaN   
                 705       IV         a        I   NaN       IV     NaN   
                 706      #iv         a        I   NaN      #iv     NaN   
                 707        V         a        I   NaN        V     NaN   

                     numeral form figbass changes relativeroot phraseend  \
    fname                                                                  
0   n01op18-1_01 168    #vii    o       7     NaN          NaN       NaN   
                 169      VI  NaN       6     NaN          NaN       NaN   
                 170       V  NaN      65     NaN           ii       NaN   
                 171    #vii    o      65     NaN          NaN       NaN   
                 172       i  NaN       6     NaN          NaN       NaN   
...                      ...  ...     ...     ...          ...       ...   
127 n15op132_05  703      vi    o      64     NaN          NaN       NaN   
                 704      vi  NaN      64     NaN          NaN       NaN   
                 705      IV  NaN     NaN     NaN          NaN       NaN   
                 706     #iv  NaN     NaN     NaN          NaN       NaN   
                 707       V  NaN     NaN     NaN          NaN       NaN   

                     chord_type  globalkey_is_minor  localkey_is_minor  \
    fname                                                                
0   n01op18-1_01 168         o7               False               True   
                 169          M               False               True   
                 170        Mm7               False               True   
                 171         o7               False               True   
                 172          m               False               True   
...                         ...                 ...                ...   
127 n15op132_05  703          o                True              False   
                 704          m                True              False   
                 705          M                True              False   
                 706          m                True              False   
                 707          M                True              False   

                         chord_tones added_tones  root  bass_note bass_name  
    fname                                             

### `i64 Ger`

In [21]:
show(ascending, ('i64', 'Ger'))

mc   mn mc_onset mn_onset timesig  staff  voice   label  \
   fname                                                                        
71 n08op59-2_01 375  201  199      1/8      1/8     6/8      1      1      I6   
                376  201  199      3/8      3/8     6/8      1      1   #io65   
                377  201  199      5/8      5/8     6/8      1      1     ii6   
                378  202  200      1/8      1/8     6/8      1      1   #ivo7   
                379  202  200      3/8      3/8     6/8      1      1     i64   
                380  202  200      5/8      5/8     6/8      1      1   .Ger6   
                381  203  201      1/8      1/8     6/8      1      1  #ivo65   

                    globalkey localkey pedal   chord special numeral form  \
   fname                                                                    
71 n08op59-2_01 375         e        I   NaN      I6     NaN       I  NaN   
                376         e        I   NaN   #io65     NaN      #i    o   
                377         e        I   NaN     ii6     NaN      ii  NaN   
                378         e        I   NaN   #ivo7     NaN     #iv    o   
                379         e        I   NaN     i64     NaN       i  NaN   
                380         e        I   NaN    Ger6     Ger     vii    o   
                381         e        I   NaN  #ivo65     NaN     #iv    o   

                    figbass changes relativeroot phraseend chord_type  \
   fname                                                                
71 n08op59-2_01 375       6     NaN          NaN       NaN          M   
                376      65     NaN          NaN       NaN         o7   
                377       6     NaN          NaN       NaN          m   
                378       7     NaN          NaN       NaN         o7   
                379      64     NaN          NaN       NaN          m   
                380      65      b3            V       NaN        Ger   
                381      65     NaN          NaN       NaN         o7   

                     globalkey_is_minor  localkey_is_minor     chord_tones  \
   fname                                                                     
71 n08op59-2_01 375                True              False       (4, 1, 0)   
                376                True              False   (4, 1, -2, 7)   
                377                True              False      (-1, 3, 2)   
                378                True              False   (6, 3, 0, -3)   
                379                True              False      (1, 0, -3)   
                380                True              False  (-4, 0, -3, 6)   
                381                True              False   (3, 0, -3, 6)   

                    added_tones  root  bass_note bass_name  
   fname                                                    
71 n08op59-2_01 375          ()     0          4        G#  
                376          ()     7          4        G#  
                377          ()     2         -1         A  
                378          ()     6          6        A#  
                379          ()     0          1         B  
                380          ()     6         -4         C  
                381          ()     6          3        C#

### `i64`

In [25]:
show(ascending, ('i64',))

mc   mn mc_onset mn_onset timesig  staff  voice  \
   fname                                                                
33 n04op18-4_02 155  106  106        0        0     3/8      1      1   
                156  107  107        0        0     3/8      1      1   
                157  108  108        0        0     3/8      1      1   
                158  109  109        0        0     3/8      1      1   
                159  110  110        0        0     3/8      1      1   
38 n04op18-4_04 97    58   58        0        0     2/2      1      1   
                98    59   59        0        0     2/2      1      1   
                99    59   59      1/2      1/2     2/2      1      1   
                100   60   60        0        0     2/2      1      1   
                101   60   60      1/4      1/4     2/2      1      1   
43 n05op18-5_01 279  157  157      3/8      3/8     6/8      1      1   
                280  158  158        0        0     6/8      1      1   
                281  158  158      3/8      3/8     6/8      1      1   
                282  159  159        0        0     6/8      1      1   
71 n08op59-2_01 375  201  199      1/8      1/8     6/8      1      1   
                376  201  199      3/8      3/8     6/8      1      1   
                377  201  199      5/8      5/8     6/8      1      1   
                378  202  200      1/8      1/8     6/8      1      1   
                379  202  200      3/8      3/8     6/8      1      1   
                380  202  200      5/8      5/8     6/8      1      1   
                381  203  201      1/8      1/8     6/8      1      1   

                              label globalkey localkey pedal      chord  \
   fname                                                                  
33 n04op18-4_02 155               I         C     bIII   NaN          I   
                156  .bvii.#viio7/V         C     bvii   NaN   #viio7/V   
                157             i64         C     bvii   NaN        i64   
                158              VI         C     bvii   NaN         VI   
                159       #viio65/V         C     bvii   NaN  #viio65/V   
38 n04op18-4_04 97                i         c        i   NaN          i   
                98             v.iv         c        v   NaN         iv   
                99         #viio7/V         c        v   NaN   #viio7/V   
                100             i64         c        v   NaN        i64   
                101              VI         c        v   NaN         VI   
43 n05op18-5_01 279              iv         A        I   NaN         iv   
                280        #viio7/V         A        I   NaN   #viio7/V   
                281             i64         A        I   NaN        i64   
                282            .bVI         A        I   NaN        bVI   
71 n08op59-2_01 375              I6         e        I   NaN         I6   
                376           #io65         e        I   NaN      #io65   
                377             ii6         e        I   NaN        ii6   
                378           #ivo7         e        I   NaN      #ivo7   
                379             i64         e        I   NaN        i64   
                380           .Ger6         e        I   NaN       Ger6   
                381          #ivo65         e        I   NaN     #ivo65   

                    special numeral form figbass changes relativeroot  \
   fname                                                                
33 n04op18-4_02 155     NaN       I  NaN     NaN     NaN          NaN   
                156     NaN    #vii    o       7     NaN            V   
                157     NaN       i  NaN      64     NaN          NaN   
                158     NaN      VI  NaN     NaN     NaN          NaN   
                159     NaN    #vii    o      65     NaN            V   
38 n04op18-4_04 97      NaN       i  NaN     NaN     NaN          NaN   
                98      NaN     

### `V(64) Ger`

In [22]:
show(ascending, ('V(64)', 'Ger'))

mc   mn mc_onset mn_onset timesig  staff  voice  \
    fname                                                                
39  n04op18-4_04 183   99   99        0        0     2/2      1      1   
                 184  100  100        0        0     2/2      1      1   
                 185  100  100      3/4      3/4     2/2      1      1   
                 186  101  101        0        0     2/2      1      1   
                 187  101  101      1/4      1/4     2/2      1      1   
40  n04op18-4_04 307  176  176        0        0     2/2      1      1   
                 308  177  177        0        0     2/2      1      1   
                 309  177  177      1/2      1/2     2/2      1      1   
                 310  178  178        0        0     2/2      1      1   
                 311  178  178      1/4      1/4     2/2      1      1   
108 n14op131_01  60    20   19      1/2      1/2     2/2      1      1   
                 61    20   19      3/4      3/4     2/2      1      1   
                 62    21   20        0        0     2/2      1      1   
                 63    21   20      1/4      1/4     2/2      1      1   

                          label globalkey localkey pedal      chord special  \
    fname                                                                     
39  n04op18-4_04 183          i         c        i   NaN          i     NaN   
                 184       v.iv         c        v   NaN         iv     NaN   
                 185   #viio7/V         c        v   NaN   #viio7/V     NaN   
                 186      V(64)         c        v   NaN      V(64)     NaN   
                 187      .Ger6         c        v   NaN       Ger6     Ger   
40  n04op18-4_04 307          i         c        i   NaN          i     NaN   
                 308       v.iv         c        v   NaN         iv     NaN   
                 309   #viio7/V         c        v   NaN   #viio7/V     NaN   
                 310      V(64)         c        v   NaN      V(64)     NaN   
                 311      .Ger6         c        v   NaN       Ger6     Ger   
108 n14op131_01  60   #viio6/iv        c#      vii   NaN  #viio6/iv     NaN   
                 61        #vo7        c#      vii   NaN       #vo7     NaN   
                 62     i.V(64)        c#        i   NaN      V(64)     NaN   
                 63       .Ger6        c#        i   NaN       Ger6     Ger   

                     numeral form figbass changes relativeroot phraseend  \
    fname                                                                  
39  n04op18-4_04 183       i  NaN     NaN     NaN          NaN       NaN   
                 184      iv  NaN     NaN     NaN          NaN       NaN   
                 185    #vii    o       7     NaN            V       NaN   
                 186       V  NaN     NaN      64          NaN       NaN   
                 187     vii    o      65      b3            V       NaN   
40  n04op18-4_04 307       i  NaN     NaN     NaN          NaN       NaN   
                 308      iv  NaN     NaN     NaN          NaN       NaN   
                 309    #vii    o       7     NaN            V       NaN   
                 310       V  NaN     NaN      64          NaN       NaN   
                 311     vii    o      65      b3            V       NaN   
108 n14op131_01  60     #vii    o       6     NaN           iv       NaN   
                 61       #v    o       7     NaN          NaN       NaN   
                 62        V  NaN     NaN      64          NaN       NaN   
                 63      vii    o      65      b3            V       NaN   

                     chord_type  globalkey_is_minor  localkey_is_minor  \
    fname                                                                
39  n04op18-4_04 183          m                True               True   
                 184          m                True               True   
                 185         o7                True         

### Bass degrees `#4 5 b6`

In [23]:
show(ascending, (6, 1, -4), 'bass_note')

mc   mn mc_onset mn_onset timesig  staff  voice  \
   fname                                                                
2  n01op18-1_02 48    19   19        0        0     9/8      1      1   
                49    19   19      3/8      3/8     9/8      1      1   
                50    19   19      3/4      3/4     9/8      1      1   
                51    20   20        0        0     9/8      1      1   
                52    21   21        0        0     9/8      1      1   
                53    21   21      7/8      7/8     9/8      1      1   
32 n04op18-4_01 494  202  200      1/2      1/2     4/4      1      1   
                495  203  201        0        0     4/4      1      1   
                496  203  201      1/2      1/2     4/4      1      1   
                497  204  202        0        0     4/4      1      1   
                498  204  202      3/4      3/4     4/4      1      1   
                499  205  203        0        0     4/4      1      1   
                500  205  203      1/4      1/4     4/4      1      1   
                501  205  203      1/2      1/2     4/4      1      1   
                502  205  203      3/4      3/4     4/4      1      1   
                503  206  204        0        0     4/4      1      1   
                504  206  204      1/4      1/4     4/4      1      1   
                505  206  204      1/2      1/2     4/4      1      1   
                506  206  204      3/4      3/4     4/4      1      1   
                507  207  205        0        0     4/4      1      1   
                508  207  205      1/4      1/4     4/4      1      1   
                509  207  205      1/2      1/2     4/4      1      1   
                510  207  205      5/8      5/8     4/4      1      1   
                511  207  205      3/4      3/4     4/4      1      1   
                512  208  206        0        0     4/4      1      1   
                513  208  206      1/2      1/2     4/4      1      1   
33 n04op18-4_02 155  106  106        0        0     3/8      1      1   
                156  107  107        0        0     3/8      1      1   
                157  108  108        0        0     3/8      1      1   
                158  109  109        0        0     3/8      1      1   
                159  110  110        0        0     3/8      1      1   
36 n04op18-4_04 6      6    6        0        0     2/2      1      1   
                7      7    7        0        0     2/2      1      1   
                8      7    7      3/4      3/4     2/2      1      1   
                9      8    8        0        0     2/2      1      1   
                10     8    8      1/4      1/4     2/2      1      1   
37 n04op18-4_04 83    50   50        0        0     2/2      1      1   
                84    51   51        0        0     2/2      1      1   
                85    51   51      3/4      3/4     2/2      1      1   
                86    52   52        0        0     2/2      1      1   
                87    52   52      1/4      1/4     2/2      1      1   
38 n04op18-4_04 97    58   58        0        0     2/2      1      1   
                98    59   59        0        0     2/2      1      1   
                99    59   59      1/2      1/2     2/2      1      1   
                100   60   60        0        0     2/2      1      1   
                101   60   60      1/4      1/4     2/2      1      1   
39 n04op18-4_04 183   99   99        0        0     2/2      1      1   
                184  100  100        0        0     2/2      1      1   
                185  100  100      3/4      3/4     2/2      1      1   
                186  101  101        0        0     2/2      1      1   
                187  101  101      1/4      1/4     2/2      1      1   
40 n04op18-4_04 307  176  176        0        0     2/2      1      1   
                308  177  177        0        0     2/2      1      1   
       